In [7]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
runtime = pd.read_parquet("../raw-data/runtimeDf.gzip")

In [10]:
stops = gpd.read_file("../data/stops/stopsGeographyProcessed.shp")
stops = stops.rename(
    columns={"directionI": "directionId", "StopId": "stopId"}
).drop("geography", axis=1)

In [32]:
routes = ["21", "33", "47"]

In [54]:
extract = (
    runtime.query("routeId.isin(@routes)")
    .copy()
    .groupby(["routeId", "directionId", "toStopId"])
    .agg({"toStopPathIndex": "min", "expectedHeadway": "mean", "DoW": "size"})
    .query("DoW > 0")
    .copy()
    .reset_index()
    .drop(["DoW"], axis=1)
)

In [49]:
stops.routeId = stops.routeId.astype(str)
stops.directionId = stops.directionId.astype(str)
stops.stopId = stops.stopId.astype(str)

In [55]:
extract = extract.merge(
    stops[["routeId", "directionId", "stopId", "centerCity"]].rename(columns={"stopId": "toStopId"}),
    how="left",
    on=["routeId", "directionId", "toStopId"],
).drop(["routeId", "directionId"], axis=1)

In [57]:
extract

,routeId,directionId,toStopId,toStopPathIndex,expectedHeadway,centerCity
0,33,0,10255,46.0,899.936776,in
1,33,0,10259,45.0,903.206349,in
2,33,0,10263,44.0,903.164557,in
3,33,0,10266,43.0,1180.823864,in
4,33,0,10272,41.0,907.301587,in
...,...,...,...,...,...,...
451,21,1,434,47.0,765.174330,post
452,21,1,434,47.0,765.174330,post
453,21,1,435,55.0,769.842400,post
454,21,1,435,55.0,769.842400,post
